# Lonboard Examples

In [1]:
import numpy as np
from matplotlib.colors import LogNorm
from palettable.colorbrewer.sequential import Oranges_9

from lonboard import Map, PolygonLayer
from lonboard.colormap import apply_continuous_cmap

In [4]:
import sedona.db
import os

os.environ["AWS_SKIP_SIGNATURE"] = "true"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

sd = sedona.db.connect()

In [5]:
df = sd.read_parquet(
    "s3://overturemaps-us-west-2/release/2025-08-20.0/theme=buildings/type=building/"
)

In [7]:
df.to_view("buildings")

In [2]:
nyc_bbox_wkt = (
    "POLYGON((-74.2591 40.4774, -74.2591 40.9176, -73.7004 40.9176, -73.7004 40.4774, -74.2591 40.4774))"
)

In [10]:
nyc = sd.sql(f"""
SELECT
    id, height, geometry, names
FROM buildings
WHERE
    ST_Intersects(
        geometry,
        ST_SetSRID(ST_GeomFromText('{nyc_bbox_wkt}'), 4326)
    )
""")

In [11]:
nyc.show(5)

┌─────────────────────────────┬─────────┬─────────────────────────────┬────────────────────────────┐
│              id             ┆  height ┆           geometry          ┆            names           │
│             utf8            ┆ float64 ┆           geometry          ┆           struct           │
╞═════════════════════════════╪═════════╪═════════════════════════════╪════════════════════════════╡
│ 98bdaaee-b58f-44e0-8295-b1… ┆     8.4 ┆ POLYGON((-74.2495739 40.50… ┆ {primary: , common: , rul… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ fb681a0b-ea9a-478b-a3e6-0c… ┆     8.3 ┆ POLYGON((-74.2495394 40.50… ┆ {primary: , common: , rul… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 66c09e01-fb69-4196-ba01-1e… ┆     8.4 ┆ POLYGON((-74.2494029 40.50… ┆ {primary: , common: , rul… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌

In [14]:
nyc_table = nyc.to_arrow_table()

In [15]:
heights = nyc_table["height"].to_numpy()
heights = np.nan_to_num(heights, nan=1)

In [16]:
normalizer = LogNorm(1, heights.max(), clip=True)
normalized_heights = normalizer(heights)
Oranges_9.mpl_colormap
colors = apply_continuous_cmap(normalized_heights, Oranges_9)

In [17]:
layer = PolygonLayer(
    table=nyc_table,
    extruded=True,
    get_elevation=heights,
    get_fill_color=colors,
)

In [18]:
view_state = {
    "longitude": -73.98416810282863,
    "latitude": 40.72651721370669,
    "zoom": 12.726630492730596,
    "pitch": 59.80465353190481,
    "bearing": 13.243243243243244,
}
m = Map(layer, view_state=view_state)
m

Map(custom_attribution='', layers=(PolygonLayer(extruded=True, get_elevation=arro3.core.ChunkedArray<Float32>
…